In [ ]:
from __future__ import absolute_import, division
from IPython.display import clear_output
from BashColors import C
from TarfileFunctions import *
from CV2_Utils_2 import *

import glob, json, os, sys
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '5'
from os.path import *
from time import perf_counter, perf_counter_ns

from PIL import Image
import numpy as np
import glob, os
from os.path import *
from scipy.ndimage import zoom
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from multiprocessing.dummy import Pool as ThreadPool

contentPath = os.getcwd()
genPath = join(contentPath, 'DataGenerator')
testPath = join(contentPath, 'images')
jsonPath = join(contentPath, 'initialGlobList.json')

removePath=join(contentPath, 'Defcon4_mobilenet_v2_Dynamic.tflite')
if exists(removePath):
    os.remove(removePath)
    print(f'{C.BIRed}{removePath}{C.ColorOff} removed')
    removePath=None

if not exists(genPath):
    tff.extractTarfiles('DataGenerator5.tar.gz')
if not exists(testPath):
    tff.extractTarfiles('images.tar.gz')

from time import perf_counter, sleep
import concurrent.futures

initialGlobList:list
with open("initialGlobList.json", 'r') as f:
    initialGlobList = json.load(f)
    
def listNewFiles(initial=initialGlobList, delete=False):
    currentFilesGlob=glob.glob('**')
    if len(initial) == len(currentFilesGlob):
        print(f'{C.BIRed}Nothing to see here.  No new files.')
    for fil in currentFilesGlob:
        if not fil in initial:
            if isdir(fil):
                print(f'{C.BIBlue}{fil}')
                if delete:
                    shutil.rmtree(fil)
            elif isfile(fil):
                print(f'{C.ColorOff}{fil}')
                if delete:
                    os.remove(fil)
listNewFiles()

In [ ]:
import numpy as np
from scipy.ndimage import zoom

def clipped_zoom(img, zoom_factor, **kwargs):
    ''' '''
    h, w = img.shape[:2]
    # For multichannel images we don't want to apply the zoom factor to the RGB
    # dimension, so instead we create a tuple of zoom factors, one per array
    # dimension, with 1's for any trailing dimensions after the width and height.
    zoom_tuple = (zoom_factor,) * 2 + (1,) * (img.ndim - 2)

    # Zooming out
    if zoom_factor < 1:
        # Bounding box of the zoomed-out image within the output array
        zh = int(np.round(h * zoom_factor))
        zw = int(np.round(w * zoom_factor))
        top = (h - zh) // 2
        left = (w - zw) // 2

        # Zero-padding
        out = np.zeros_like(img)
        out[top:top+zh, left:left+zw] = zoom(img, zoom_tuple, **kwargs)

    # Zooming in
    elif zoom_factor > 1:
        # Bounding box of the zoomed-in region within the input array
        zh = int(np.round(h / zoom_factor))
        zw = int(np.round(w / zoom_factor))
        top = (h - zh) // 2
        left = (w - zw) // 2

        out = zoom(img[top:top+zh, left:left+zw], zoom_tuple, **kwargs)

        # `out` might still be slightly larger than `img` due to rounding, so
        # trim off any extra pixels at the edges
        trim_top = ((out.shape[0] - h) // 2)
        trim_left = ((out.shape[1] - w) // 2)
        out = out[trim_top:trim_top+h, trim_left:trim_left+w]

    # If zoom_factor == 1, just return the input array
    else:
        out = img
    return out

In [ ]:
path=join(contentPath, 'Fighter Back 2.png')
img = cvu.getCV2Image(path)
img = cvu.resizeImage(img)
print(img.shape)
img = img[:,:,::-1]

In [ ]:
zm1 = clipped_zoom(img, 0.75)
zm2 = clipped_zoom(img, 0.5)
zm3 = clipped_zoom(img, 0.25)

fig, ax = plt.subplots(1, 4)
ax[0].imshow(img)
ax[1].imshow(zm1)
ax[2].imshow(zm2)
ax[3].imshow(zm3)

In [ ]:
zm1 = clipped_zoom(img, 1.75)
zm2 = clipped_zoom(img, 1.5)
zm3 = clipped_zoom(img, 1.25)

fig, ax = plt.subplots(1, 4)
ax[0].imshow(img)
ax[1].imshow(zm1)
ax[2].imshow(zm2)
ax[3].imshow(zm3)